In [2]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import datetime
import re
import time
import os
import json
from PIL import Image 
import PIL 
pd.set_option('display.max_columns', None)
import random
import codecs
import subprocess
from selenium import webdriver
from bs4 import BeautifulSoup
import chromedriver_binary
from selenium.webdriver.chrome.options import Options

# Headers

In [3]:
headers = {
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
    'sec-ch-ua-mobile': '?0',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-User': '?1',
    'Sec-Fetch-Dest': 'document',
    'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
}

# Save pages

In [5]:
def save_page_list(req, page=1):
    datetime_1 = datetime.datetime.now().strftime("%Y-%m-%d_%Hh%M")
    page_list_name = "fulloccaz" + "-" + datetime_1 + "-" + str(page)
    with open("pages/" + page_list_name + ".html", "w")  as file:
        file.write(req.text)
        file.close()
    return "pages/" + page_list_name + ".html"

In [5]:
#def get_source_selenium(url_):
#    options = Options()
#    options.add_argument("--headless")
#    driver = webdriver.Chrome(options=options, executable_path="/Users/victor/Downloads/chromedriver")
#    driver.get(url_)
#    page_source = driver.page_source
#    driver.quit()
#    return page_source

In [ ]:
scrap = 1
page_=1
#
while scrap==1:
    html_file = requests.get(f"https://www.fulloccaz.com/?&page={page_}", headers=headers)
    last_saved_file = save_page_list(html_file, page=page_)
    
    with open(last_saved_file, 'r') as f:
        readable_html = f.read()
        prices = get_prices_from_pages(readable_html)
    if len(prices) == 0:
        scrap=0

    page_ += 1    
    time.sleep(random.randint(1, 3))
    print("page saved: " + str(page_))

# Extract data from pages

## Test file

In [8]:
file_test_name = [file for file in os.listdir("pages") if file.endswith("html")][-2]
file_test_name

'fulloccaz-2021-06-05_18h03-4.html'

In [9]:
file_check_x15_good = f"pages/{file_test_name}"
file_check_x15_good

'pages/fulloccaz-2021-06-05_18h03-4.html'

## Get prices from pages

In [12]:
def get_prices_from_pages(r):
    
    # set empty price list
    price_list = []
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract price
    step1 = soup.select('div[id*="IDVignettes"]')
    if len(step1) > 0:
        step2 = step1[0].select('div[class*="CLPrix"]')
        if len(step2) > 0:
            for k in step2:
                try:
                    price_list.append(float("".join(re.findall("[0-9]", k.text.replace("\n", "").replace("\t", "")))))
                except:
                    price_list.append(np.nan)    
    return price_list

In [14]:
with open(file_check_x15_good, 'r') as f:
    readable_html = f.read()
    prices = get_prices_from_pages(readable_html)
prices

[3999.0,
 3500.0,
 13990.0,
 10690.0,
 4500.0,
 3490.0,
 1600.0,
 6400.0,
 1500.0,
 4699.0,
 15990.0,
 9990.0]

## Get url from pages

In [28]:
def get_urls_from_pages(r):
    
    # set empty url list
    url_list = []
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract price
    step1 = soup.select('div[id*="IDVignettes"]')
    if len(step1) > 0:
        step2 = step1[0].select('a')
        if len(step2) > 0:
            for k in step2:
                if "href" in k.attrs:
                    try:
                        url_list.append("https://www.fulloccaz.com" + k.attrs["href"])
                    except:
                        url_list.append(np.nan)    
    return url_list

In [29]:
with open(file_check_x15_good, 'r') as f:
    readable_html = f.read()
    urls = get_urls_from_pages(readable_html)
urls

['https://www.fulloccaz.com/fiches/index/2016520-occasion-honda-cbf-1000-f?',
 'https://www.fulloccaz.com/fiches/index/2037451-occasion-honda-cb-600-f-hornet?',
 'https://www.fulloccaz.com/fiches/index/2036585-occasion-moto-guzzi-california?',
 'https://www.fulloccaz.com/fiches/index/2021545-occasion-ktm-790-adventure?',
 'https://www.fulloccaz.com/fiches/index/2013814-occasion-suzuki-gsf-bandit-1250-s?',
 'https://www.fulloccaz.com/fiches/index/2036850-occasion-yamaha-majesty-400?',
 'https://www.fulloccaz.com/fiches/index/2036206-occasion-yamaha-x-max-125?',
 'https://www.fulloccaz.com/fiches/index/2046628-occasion-peugeot-metropolis-400?',
 'https://www.fulloccaz.com/fiches/index/2036168-occasion-yamaha-majesty-125?',
 'https://www.fulloccaz.com/fiches/index/2022748-occasion-vespa-gts-super-sport?',
 'https://www.fulloccaz.com/fiches/index/2015330-occasion-triumph-bonneville-1200-bobber?',
 'https://www.fulloccaz.com/fiches/index/2015314-occasion-triumph-street-triple-rs?']

## Get references from pages

In [36]:
def get_references_from_url_list(url_list_):
    
    # set empty ref list
    ref_list = []
    
    for k in url_list_:
        try:
            ref_list.append(k.split("/")[5].split("-")[0])
        except:
            ref_list.append(np.nan)
    #                
    return ref_list

In [37]:
get_references_from_url_list(urls)

['2016520',
 '2037451',
 '2036585',
 '2021545',
 '2013814',
 '2036850',
 '2036206',
 '2046628',
 '2036168',
 '2022748',
 '2015330',
 '2015314']

# Download single announce pages

In [24]:
data_motoplanete = "../../../tresboncoin/data/motoplanete.csv"
#data = pd.read_csv(data_motoplanete)

In [25]:
def add_the_announce(df, uniq_id, price):
    if int(uniq_id) in list(df["unique id"]):
        index_ = df.index[df['unique id'] == int(uniq_id)].tolist()[0]
        
        # if an announce with same uniq id and same price is found, return true and skip
        return df.iloc[index_]["price"]!=price
    
    # else, return false and add the announce
    return True

In [26]:
def save_page_uniq(req, uniq_id):
    datetime_1 = datetime.datetime.now().strftime("%Y-%m-%d_%Hh%M")
    page_name = "motoplanete" + "-" + uniq_id + "-" + datetime_1
    with open("annonces/" + page_name + ".html", "w")  as file:
        file.write(req.text)
        file.close()

In [27]:
def file_exists(uniq_id):
    for k in os.listdir("annonces"):
        if "-" + str(uniq_id) + "-" in k:
            return True
    return False

In [ ]:
page_count = 1
for html_file in [file for file in os.listdir("pages") if file.endswith("html")]:
    with open(f"pages/{html_file}", 'r') as f:
        readable_html = f.read()
        urls_list = get_urls_from_pages(readable_html)
        unique_id = get_references_from_url_list(urls_list)
        prices_list = get_prices_from_pages(readable_html)
    if urls_list != None:
        for url_single, uniq_id, price_ in zip(urls_list, unique_id, prices_list):
            
            #if add_the_announce(data_price, uniq_id, price_):
            #html_file = get_source_selenium(url_single)
            if file_exists(uniq_id) == False:
                req_uniq = requests.get(url_single, headers = headers)
                save_page_uniq(req_uniq, uniq_id)
                print(str(page_count) + ". saved page id: " + uniq_id)
                time.sleep(random.randint(1, 5))
                page_count += 1
            else:
                print(url_single + " --> skipped")

# Process single announces
Saving scaled images x3 and adding announce to DataFrame

In [350]:
announce_template = pd.DataFrame({"url": [np.nan],
                                  "unique id": [np.nan],
                                  "date_scrapped": [np.nan],
                                  "announce_publication_date": [np.nan],
                                  "vehicle brand": [np.nan],
                                  "vehicle type": [np.nan],
                                  "color": [np.nan],
                                  "hand": [np.nan],
                                  "price": [np.nan],
                                  "city": [np.nan],
                                  "postal code": [np.nan],
                                  "seller": [np.nan],
                                  "seller_name": [np.nan],
                                  "vehicle release date": [np.nan],
                                  "mileage": [np.nan],
                                  "engine capacity [CC]": [np.nan],
                                  "comments": [np.nan]})
announce_template

,url,unique id,date_scrapped,announce_publication_date,vehicle brand,vehicle type,color,hand,price,city,postal code,seller,seller_name,vehicle release date,mileage,engine capacity [CC],comments
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## test announce

In [34]:
file_name = os.listdir("annonces")[-1]
#
file_check_single = f"annonces/{file_name}"
file_check_single

'annonces/motoplanete-41986-2021-06-03_23h18.html'

## Functions

### Get url

In [36]:
def get_url(r):
    
    # set empty url list
    url_ = ""
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract url
    step1 = soup.select('meta[property*="og:url"]')
    if len(step1)>0:
        return step1[0].attrs["content"].split("?")[0]
    else:
        return None

In [37]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_url = get_url(readable_html)
g_url

'https://www.motoplanete.com/occasion-moto/41986/HPS/fiche.html'

### Get unique id

In [52]:
def get_uniq_id(url_):
    return url_.split("occasion-moto")[1].split("/")[1]

In [53]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_url = get_url(readable_html)
get_uniq_id(g_url)

'41986'

### get publication date

In [128]:
def get_publication_date(r):
    
    # set empty 
    pub_ = ""
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('ul[class*="details"]')
    if len(step1)>0:
        for k in step1[0].select("li"):
            if k.select('i')[0].attrs["class"][0] == "icon-profile":
                return k.select("b")[0].text
    else:
        return np.nan

In [129]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_pub = get_publication_date(readable_html)
g_pub

'04/08/2020'

### Get vehicule brand

In [143]:
def get_brand_and_model(r):
    
    # set empty 
    brand_ = ""
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('h1')
    if len(step1)>0:
        return step1[0].text
    else:
        return None

In [144]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_brand = get_brand_and_model(readable_html)
g_brand

'FB Mondial HPS 2020'

### Get vehicule type

In [146]:
def get_type(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('ul[class*="details"]')
    if len(step1)>0:
        for k in step1[0].select("li"):
            if k.select('i')[0].attrs["class"][0] == "icon-roadster-2":
                return k.select("b")[0].text
    else:
        return np.nan

In [147]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_type = get_type(readable_html)
g_type

'Roadster'

### get color

In [156]:
def get_color(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('ul[class*="details"]')
    if len(step1)>0:
        for k in step1[0].select("li"):
            if k.select('i')[0].attrs["class"][0] == "icon-format_color_fill":
                return k.select("b")[0].attrs["class"][-1]
    else:
        return np.nan

In [157]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_type = get_color(readable_html)
g_type

'noir'

### get hand

In [163]:
def get_hand(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('ul[class*="details"]')
    if len(step1)>0:
        for k in step1[0].select("li"):
            if k.select('i')[0].attrs["class"][0] == "icon-user":
                return k.text
    else:
        return np.nan

In [165]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_type = get_hand(readable_html)
g_type

'Première main'

### get price

In [178]:
def get_price(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('ul[class*="details"]')
    if len(step1)>0:
        for k in step1[0].select("li"):
            if k.select('i')[0].attrs["class"][0] == "icon-eur":
                try:
                    return float("".join(re.findall("[0-9]", k.select("b")[0].text)))
                except:
                    return np.nan
    else:
        return np.nan

In [179]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_type = get_price(readable_html)
g_type

3490.0

### get city

In [198]:
def get_city(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="contact"]')
    if len(step1)>0:
        try:
            return step1[0].select("span")[-1].text
        except:
            return np.nan
    return None

In [199]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_city = get_city(readable_html)
g_city

'ROCHEFORT DU GARD'

### get postal code

In [201]:
def get_postal(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="contact"]')
    if len(step1)>0:
        try:
            return step1[0].select("span")[0].text
        except:
            return np.nan
    return None

In [202]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_post = get_postal(readable_html)
g_post

'30650'

### get seller

In [243]:
def get_seller(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # count
    count_ = 0
    
    # extract
    step1 = soup.select('div[class*="contact"]')
    if len(step1)>0:
        step2 = step1[0].select('p')
        while step2[count_].text != "Concession":
            count_+=1
        try:
            return step1[0].select('p')[count_+1].text
        except:
            return np.nan
    return None

In [244]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_seller = get_seller(readable_html)
g_seller

'TRIUMPH PROVENCE'

### get seller name

In [246]:
def get_seller_name(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # count
    count_ = 0
    
    # extract
    step1 = soup.select('div[class*="contact"]')
    if len(step1)>0:
        step2 = step1[0].select('p')
        while step2[count_].text != "Contact":
            count_+=1
        try:
            return step1[0].select('p')[count_+1].text
        except:
            return np.nan
    return None

In [247]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_seller_name = get_seller_name(readable_html)
g_seller_name

'Nicolas'

### get vehicle release date

In [253]:
def get_release(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('ul[class*="details"]')
    if len(step1)>0:
        for k in step1[0].select("li"):
            if k.select('i')[0].attrs["class"][0] == "icon-profile":
                try:
                    return k.select("b")[0].text
                except:
                    return np.nan
    else:
        return np.nan

In [254]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_rel = get_release(readable_html)
g_rel

'04/08/2020'

### get vehicle mileage

In [258]:
def get_mileage(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('ul[class*="details"]')
    if len(step1)>0:
        for k in step1[0].select("li"):
            if k.select('i')[0].attrs["class"][0] == "icon-meter":
                try:
                    return float("".join(re.findall("[0-9]", k.select("b")[0].text)))
                except:
                    return np.nan
    else:
        return np.nan

In [259]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_mileage = get_mileage(readable_html)
g_mileage

2000.0

### get engine capacity

In [261]:
def get_capa(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('ul[class*="details"]')
    if len(step1)>0:
        for k in step1[0].select("li"):
            if k.select('i')[0].attrs["class"][0] == "icon-picto-pieces-moteur":
                try:
                    return float("".join(re.findall("[0-9]", k.select("b")[0].text)))
                except:
                    return np.nan
    else:
        return np.nan

In [262]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_capa = get_capa(readable_html)
g_capa

125.0

### get comment

In [302]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', raw_html)
    return cleantext

In [357]:
def get_comment(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="description"]')
    if len(step1)>0:
        try:
            return cleanhtml(step1[0].text.replace("\n", " ").replace("\t", " ").replace("  ", "").strip())
        except:
            return ""
    return ""

In [358]:
with codecs.open(file_check_single, 'r', encoding='utf-8', errors='ignore') as f:
    readable_html = f.read()
    g_comment = get_comment(readable_html)
g_comment

'Mot du vendeur : FB MONDIAL 125Moto propre, entretenue et soignée (Carnet d\'entretien + Manuel d\'utilisation disponibles)Cette moto est une première mainAucuns frais à prévoirGarantie 6 moisD\'origine est équipée :* ABS * Plaque Latérale_____________________________FINANCEMENT partiel ou intégral.EXTENSION de Garantie.EXPEDITION dans toute la France.ASSURANCE sur place.REPRISE possible."Les données présentées dans cette annonce peuvent être sujettes à des erreurs. Pour toute information, contactez nous par téléphone ou par mail." ,1ère main,Disponible à l\'essai,Garantie 6 mois'

### Save image functions

In [336]:
def get_images(r, uniq_id):
    
    # get images url list
    image_list = []
    
    # get soup
    soup_ = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup_.select('img[class*="swiper-lazy"]')
    
    if len(step1)>0:
        for k in step1:
            if "data-src" in k.attrs:
                image_list.append("https://www.motoplanete.com" + k.attrs['data-src'])
    
    # save images
    k=1
    for image_url in image_list[0:3]:
        image_name = f'images/{uniq_id}-{k}.jpg'
        if os.path.isfile(image_name) is False:
            img_data = requests.get(image_url).content
            with open(image_name, 'wb') as handler:
                handler.write(img_data)
            try:
                image = Image.open(image_name) 
                ratio = image.size[0] / image.size[1]
                image = image.resize((300,int(300/ratio)))
                image.save(f'images/{uniq_id}-{k}.jpg',optimize = True, quality = 70)
            except:
                pass
        k+=1
        
    return

In [337]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_url = get_url(readable_html)
    get_images(readable_html, get_uniq_id(g_url))

<div style="color:orangered; font-weight:700; font-size:20pt">TEST 100 ANNONCES</div>

In [ ]:
counter = 1
list_test = []
for single_ in os.listdir("annonces")[0:100]:
    with open(f"annonces/{single_}", 'r') as f:
        readable_html = f.read()
        g_X = get_comment(readable_html)
        print(counter, g_X)
        counter+=1

## Extract 1 announce

In [378]:
def process_announce(file_name):
    """ this function add an announce content from motoplanete.com to the dataframe """
    
    # load csv if exists or starting from template
    data_motoplanete = "../../../tresboncoin/data/motoplanete.csv"
    #
    if os.path.isfile(data_motoplanete) is False:
        pass
    else:
        data = pd.read_csv(data_motoplanete)
    
    df = announce_template.copy()
    
    # fill df with defined functions
    with open(f"annonces/{file_name}", 'r') as f:
        readable_html = f.read()
        
        df["url"] = get_url(readable_html)
        df["unique id"] = get_uniq_id(get_url(readable_html))
        df["date_scrapped"] = datetime.datetime.now().strftime("%Y/%m/%d - %Hh%M")
        df["announce_publication_date"] = get_publication_date(readable_html)
        df["vehicle brand"] = get_brand_and_model(readable_html)
        df["vehicle type"] = get_type(readable_html)
        df["color"] = get_color(readable_html)
        df["hand"] = get_hand(readable_html)
        df["price"] = get_price(readable_html)
        df["city"] = get_city(readable_html)
        df["postal code"] = get_postal(readable_html)
        df["seller"] = get_seller(readable_html)
        df["seller_name"] = get_seller_name(readable_html)
        df["vehicle release date"] = get_release(readable_html)
        df["mileage"] = get_mileage(readable_html)
        df["engine capacity [CC]"] = get_capa(readable_html)
        df["comments"] = get_comment(readable_html)
    
        # save images
        get_images(readable_html, get_uniq_id(get_url(readable_html)))
        
    # concatenate to csv and write
    try:
        data = pd.concat([data, df], axis=0)
        data.to_csv(path_or_buf = data_motoplanete, index=False)
    except:
        df.to_csv(path_or_buf = data_motoplanete, index=False)
    
    # deplacer le fichier html traité dans le vault
    subprocess.run(["mv", f"annonces/{file_name}", "vault"])
    return

In [379]:
list(os.listdir("annonces")[0:2])

['.DS_Store', 'motoplanete-41722-2021-06-04_20h02.html']

In [15]:
for single_ in [file for file in os.listdir("annonces") if file.endswith(".html")]:
    process_announce(single_)

In [16]:
data_moto = "../../../tresboncoin/data/motoplanete.csv"
data = pd.read_csv(data_moto)
data.tail(2)

,url,unique id,date_scrapped,announce_publication_date,vehicle brand,vehicle type,color,hand,price,city,postal code,seller,seller_name,vehicle release date,mileage,engine capacity [CC],comments
2717,https://www.motoplanete.com/occasion-moto/4271...,42718,2021/06/05 - 09h00,24/07/2013,Kawasaki ER-6 2013,Roadster,argent,NaN,4500.0,RUAUDIN,72230,LE MANS MOTO,Service commercial,24/07/2013,20500.0,650.0,"Mot du vendeur : ,Compatible Permis A2,Garanti..."
2718,https://www.motoplanete.com/occasion-moto/4066...,40665,2021/06/05 - 09h00,20/07/2018,BMW K 1600 GTL 2018,Routière & GT,blanc,Première main,20800.0,AVIGNON,84000,JMS YAM STORE,Service commercial,20/07/2018,16213.0,1600.0,Mot du vendeur : BMW K 1600 GTL 20/07/2018 JMS...


In [5]:
data.shape

(2719, 17)